### 2.0 Refazendo o código

#### Baseado em ETL - Extração, Transformação e Carga (Extraction, Transformation and Load)
#### ETL - Extraction, Load and Transformation (DW)

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import plotly.express as px

# ----------------------------------------------
def show_dtypes( data ):
    print( data.dtypes )
    return None

def show_dimensions(data):
    print(f"Number of rows: {df1.shape[0]}")
    print(f"Number of columns: {df1.shape[1]}")
    return None

def collect_geodata( data, cols ):

    # initialize API
    geolocator = Nominatim(user_agent='geopiExercises')

    # Create empty columns
    data.loc[:, cols[0]] = 'NA'
    data.loc[:, cols[1]] = 'NA'

    for i in range(len(data)):
        print(f'Loop: {i}/{len(data)}')

        query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
        response = geolocator.reverse(query)

        if cols[0] in response.raw['address']:
            data.loc[i, 'house_number'] = response.raw['address'][cols[0]]

        if cols[1] in response.raw['address']:
            data.loc[i, 'road'] = response.raw['address'][cols[1]]
    
    return data


# ----------------------------------------------
data = pd.read_csv("datasets\kc_house_data.csv")

show_dimensions(data)

show_dtypes(data)

# ----------------------------------------------
data['date'] = pd.to_datetime(data['date'])

# Descriptive statistics
num_attributes = data.select_dtypes(include=['int64', 'float64'])

# Central tendency  - media, mediana
pd.set_option('display.float_format', lambda x: '%.3f' % x)

media = pd.DataFrame(num_attributes.apply(np.mean))
mediana = pd.DataFrame(num_attributes.apply(np.median))

# Dispersion - stf, min, max
std = pd.DataFrame(num_attributes.apply(np.std))
min_ = pd.DataFrame(num_attributes.apply(np.min))
max_ = pd.DataFrame(num_attributes.apply(np.max))

df1 = pd.concat([max_, min_, media, mediana, std], axis=1).reset_index()
df1.columns = ['attributes', 'maximo', 'minimo', 'media', 'mediana', 'std']

show_dimensions(df1)

data['dormitory_type'] = data['bedrooms'].apply(lambda x: 'studio' if x == 1 else
                                                'apartment' if x == 2 else 'house')

data['level'] = data['price'].apply(lambda x: 0 if x < 321950 else
                                    1 if (x > 321950) & (x < 450000) else
                                    2 if (x > 450000) & (x < 645000) else 3)

show_dimensions(data)

# ----------------------------------------------

houses = data[['id', 'lat', 'long', 'price', 'level']].copy()

fig = px.scatter_mapbox(houses,
                        lat='lat',
                        lon='long',
                        color='level',
                        size='price',
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        zoom=10)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(height=600, margin={'r': 0, 'l': 0, 'b': 0, 't': 0})
